# Analysing Smartphone Prices

In [1]:
from statistics import stdev, median

import requests
from lxml import html
from urllib import parse
import re

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

## 1. Loading Devices Names

In [2]:
RELATIVE_PATH_TO_DATA_FOLDER = './data/'
filename = 'devices_using_ribonapp_data'
filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + filename + '.csv'
devices_df = pd.read_csv(filename_with_path, skiprows=1, names=['model_name', 'old_price_range'])
devices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129 entries, 0 to 1128
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   model_name       1129 non-null   object
 1   old_price_range  1129 non-null   object
dtypes: object(2)
memory usage: 17.8+ KB


### Cleaning Duplicates and Missing Values

In [3]:
devices_df.drop_duplicates(['model_name'], inplace=True)
devices_df.dropna(axis=0, inplace=True)
devices_df

,model_name,old_price_range
0,2 A2005,Low
1,3,Low
2,3T,Low
3,4047D,Low
4,5,Low
...,...,...
1124,nova 4,Low
1125,t1_8p0lte,Low
1126,vivo 1724,Low
1127,vivo X9,Low


### Filtering Non Informative Names

In [4]:
informative_names_mask = devices_df['model_name'].str.len() > 1
devices_df = devices_df[informative_names_mask]
devices_df

,model_name,old_price_range
0,2 A2005,Low
2,3T,Low
3,4047D,Low
5,5051J,Low
6,5056D,Low
...,...,...
1124,nova 4,Low
1125,t1_8p0lte,Low
1126,vivo 1724,Low
1127,vivo X9,Low


# Searching for Prices

In [7]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MercadoLivreSpider)
process.start() # the script will block here until the crawling is finished

2020-05-18 17:25:54 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-05-18 17:25:54 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 20.3.0, Python 3.6.9 (default, Nov  7 2019, 10:44:02) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-5.3.0-51-generic-x86_64-with-Ubuntu-18.04-bionic
2020-05-18 17:25:54 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-05-18 17:25:54 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-05-18 17:25:54 [scrapy.extensions.telnet] INFO: Telnet Password: 65706df12297a629
2020-05-18 17:25:54 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-05-18 17:25:54 

In [ ]:
MercadoLivreSpider.

In [ ]:
extracted_data_df = pd.DataFrame(data=found_elements, columns=['item_original_name','url', 'improved_url', 'devices_names', 'devices_prices'])
extracted_data_df

In [ ]:


url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()
url_list = [url_base + parse.quote(item_name) for item_name in item_list]

# We can use the website filtering to help our search
# Let's filter for only devices in the tag 'Novo'
novo_filter_xpath = '/html/body/main/div[2]/div/aside/section[2]/dl[@id="id_ITEM_CONDITION"]/dd[1]/a/@href'
products_articles_criteria = '@class="results-item highlighted article stack product "'
product_information_criteria = '@class="item__info-container highlighted "'
item_prices_criteria = '@class="price__container"'
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'
headers = {'User-Agent': user_agent}

found_elements = []
for i, url in tqdm(enumerate(url_list)):
    webpage_requested = requests.get(url, headers=headers, allow_redirects=True)
    if not webpage_requested or not webpage_requested.content:
        found_elements.append([item_list[i], url, [], [], []])
        continue

    webpage_tree = html.fromstring(webpage_requested.content)


    improved_url = webpage_tree.xpath(novo_filter_xpath)
    if improved_url:
        improved_url = improved_url[0]
        webpage_requested = requests.get(improved_url)
        webpage_tree = html.fromstring(webpage_requested.content)

    
    webproducts_names_xpath = '/html/body/main/div[2]/div/section/ol/li[' + products_articles_criteria + ']/div/div[' + product_information_criteria + ']/div/h2/a/span/text()'
    webproducts_prices_xpath = '/html/body/main/div[2]/div/section/ol/li[' + products_articles_criteria +\
        ']/div/div['+ product_information_criteria + ']/div/div[' + item_prices_criteria + ']/div/span[2]/text()'

    webproducts_names_found = webpage_tree.xpath(webproducts_names_xpath)
    webproducts_prices_found = webpage_tree.xpath(webproducts_prices_xpath)
    if len(webproducts_names_found) > len(webproducts_prices_found):
        webproducts_names_found = webproducts_names_found[:len(webproducts_prices_found)]
    elif len(webproducts_names_found) < len(webproducts_prices_found = ):
        webproducts_prices_found = webproducts_prices_found[:len(webproducts_names_found)]
    
    found_elements.append([item_list[i], url, improved_url, webproducts_names_found, webproducts_prices_found])
print(len(found_elements))

In [ ]:
print(found_elements)

In [ ]:
# Explode/Split column into multiple rows
new_df = pd.DataFrame(df.City.str.split('|').tolist(), index=df).stack()
new_df = new_df.reset_index([0, 'EmployeeId'])
new_df.columns = ['EmployeeId', 'City']

In [ ]:
url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()[:5]

# TODO ADD OPTION TO SEARCH ONLY NEW PRODUCTS 
# title="Novo" href="(.*?)">
url_list = [url_base + parse.quote(item_name) for item_name in item_list]
found_search_contents = []

In [ ]:
url_base = "https://celulares.mercadolivre.com.br/"
item_list = devices_df['model_name'].to_list()

# TODO ADD OPTION TO SEARCH ONLY NEW PRODUCTS 
# title="Novo" href="(.*?)">
url_list = [url_base + parse.quote(item_name) for item_name in item_list]
found_search_contents = []
item_count = 1
for url, item_original_name in zip(url_list, item_list):
    print(item_count)
    
    item_count += 1
    r = requests.get(url)
    text_content = str(r.text)
    # We can use the website filtering to help our search
    # Let's filter for only devices in the tag 'Novo'
    improved_url = re.search(
             r"""id="id_ITEM_CONDITION" role="listbox">[\s\S]+?title=['"]Novo['"] href=['"](\S+)['"]\s*>""", text_content)
    if improved_url:
        improved_url = improved_url.groups()[0]
        r = requests.get(improved_url)
        text_content = str(r.text)
     
    # Let's filter the are to search for only search results

    result_field_text_content = re.search(r'<ol id="searchResults"(.+?)<\/ol>', text_content)
    if result_field_text_content:
        result_field_text_content = result_field_text_content.groups()[0]
    else:
        result_field_text_content = None
    found_search_contents.append([item_original_name, url, result_field_text_content])

extracted_data_df = pd.DataFrame(data=found_search_contents, columns=['item_original_name','url', 'page_search_content'])
extracted_data_df

## Storing Raw Price Information

In [ ]:
extracted_filename = 'device_search_results'
extracted_filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + extracted_filename + '.csv'

In [ ]:
extracted_data_df.to_csv(extracted_filename_with_path, index=False)

## Treating Data

In [ ]:
device_raw_extracted_data_df = pd.read_csv(extracted_filename_with_path, header=0)
device_raw_extracted_data_df.info()

In [ ]:
%time
device_raw_extracted_data_df.dropna(axis=0, inplace=True)
device_raw_extracted_data_df

In [ ]:
INVALID_WORDS = set(['DEFEITO', 'TROCO', 'QUEBRADO', 'CABO', 'CONECTOR', 'PEÇA'])
pages_contents = device_raw_extracted_data_df
number_of_items_to_count = 10
item_names_and_prices = []
for i, device_row in device_raw_extracted_data_df.iterrows():
    page_content = device_row['page_search_content']
    item_original_name = device_row['item_original_name']
    
    raw_names_and_prices = re.findall(r'"main-title">(.+?)<\/span>[\s\S]*?"price__fraction">([\d\.]+?)<\/span>', page_content)

    valid_prices = []
    choosen_name = None
    for item_raw_name, item_raw_price in raw_names_and_prices:
        found_name = item_raw_name.upper()
        if not INVALID_WORDS in set(found_name) or item_original_name.upper() in found_name:
            if not choosen_name:
                choosen_name = found_name
            item_price = None
            if '.' in item_raw_price:
                # In this case there are more than one dot symbol
                item_price = 0
                raw_price_numbers_list = item_raw_price.split('.')
                for i, number in enumerate(raw_price_numbers_list):
                    item_price += int(number)*10**((len(raw_price_numbers_list)-i-1)*3)
                item_price = int(item_price)
            else:
                item_price = int(item_raw_price)
            valid_prices.append(item_price)

    if valid_prices:
        item_price = int(median(valid_prices))
    else:
        item_price = None

    item_names_and_prices.append([item_original_name, choosen_name, item_price])
print(len(item_names_and_prices))


In [ ]:
extracted_data_df = pd.DataFrame(data=item_names_and_prices, columns=['item_original_name','item_extracted_name', 'item_price'])
print(extracted_data_df.info())
extracted_data_df

##  Storing Results

In [ ]:
extracted_filename = 'extracted_device_data_2'
extracted_filename_with_path = RELATIVE_PATH_TO_DATA_FOLDER + extracted_filename + '.csv'

In [ ]:
extracted_data_df.to_csv(extracted_filename_with_path, index=False)

#  Analysing Prices

In [ ]:
device_extracted_data_df = pd.read_csv(extracted_filename_with_path, header=0)
device_extracted_data_df.info()

## Prices Distribution

In [ ]:
device_extracted_data_df.sort_values(by='item_price', inplace=True)
device_extracted_data_df.plot.scatter(x=0, y='item_price', figsize=(12, 12))

## Prices Binning

In [ ]:
device_extracted_data_df.describe()
device_extracted_data_df.plot.hist(by=['item_price'], bins=5, figsize=(8, 8))


# Price  Ranges
![Brazilian IBGE Salaries Ranges Research](./assets/ibge_brazilian_salary_ranges.jpg)

In [ ]:
ibge_price_ranges_labels = ['very_low', 'low', 'medium', 'high', 'very_high']
salaries_ibge_ranges = [(0, 1908), (1909, 2862), (2863, 5724), (5725, 3566)]
ibge_price_ranges = [0]
for i, salarie_range in enumerate(salaries_ibge_ranges):
    threshold_price = (salarie_range[0] + salarie_range[1])//3
    # As the values are so low, we have to increment them with the previous threshold
    threshold_price += ibge_price_ranges[i-1]
    ibge_price_ranges.append(threshold_price)
ibge_price_ranges.append(float('inf'))

device_extracted_data_df['ibge_price_range'] = pd.cut(device_extracted_data_df['item_price'], bins=ibge_price_ranges, labels=ibge_price_ranges_labels)
groups = device_extracted_data_df.groupby('ibge_price_range', sort=False)
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group['item_original_name'], group['item_price'], marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()

In [ ]:
device_extracted_data_df

## Comparing with old price ranges

In [ ]:
device_extracted_data_df = device_extracted_data_df.merge(right=devices_df, left_on='item_original_name', right_on='model_name', how='inner').drop(columns='model_name')
groups = device_extracted_data_df.groupby('old_price_range', sort=False)
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group['item_original_name'], group['item_price'], marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()